In [ ]:
%run ../commons/imports.ipynb
%run ../commons/helper_db.ipynb
%run ../commons/helper_contents.ipynb

In [ ]:
MAIN_FOLDER = main_folder("bluesky")

create_database('blp_dataset')
create_table('products_tmp_bluesky')

In [ ]:
folders = [x for x in Path(MAIN_FOLDER).iterdir() if x.is_dir()]

for fo in folders:
    folder = Path(f"{fo}\product\\view")
    folder_image = Path(f"{fo}")

    if folder.exists():
        for file in os.listdir(folder):
            page_content = BeautifulSoup(open(Path(f"{folder}\{file}"), encoding='utf8').read())

            if page_content is not None:
                ## GET DESCRIPTION
                description = ""
                raw_desc = page_content.find("div", attrs={"style": "margin-top:0px; "})

                if raw_desc is not None:
                    description = get_content(raw_desc)

                    if description is not None:
                        description = description.replace("Description: ", "")

                ## GET TITLE
                title = ""
                raw_title = page_content.find("div", attrs={"class": "prod_title"})
                
                if raw_title is not None:
                    title = get_content(raw_title)
                    
                if description != "" or title != "":
                    # Insert product into database
                    query = "INSERT INTO products_tmp_bluesky (name, description, market_name, illegal) VALUES (%s, %s, %s, %s)"
                    values = (title, description, "Blue Sky", "t")

                    cursor.execute(query, values)
                    conn.commit()

                    product_id = cursor.lastrowid

                    ## GET IMAGE
                    img_content = page_content.find("img")

                    if img_content is not None:
                        image = img_content["src"]

                        try:
                            data = "" + image.replace("data:image/jpeg;base64,", "")
                            imgdata = base64.b64decode(data)

                            img_filename = Path(f"D:\images\_bluesky\{product_id}.jpeg")

                            with open(img_filename, 'wb') as f:
                                f.write(imgdata)
                                f.close
                        except:
                            print(f"Image {product_id} has error. Except")
                            print(f"{folder_image}\{file}")
                            update_image(product_id, "no_image")
                    else:
                        print(f"Product {product_id} has no image")
                        update_image(product_id, "no_image")